# **Phân Tích Dữ Liệu Điểm Thi THPT Quốc Gia 2020**

**Mục tiêu:** Thực hành các chức năng chính của chuẩn bị dữ liệu như: xử lý dữ liệu thiếu, dữ liệu ngoại lệ, thống kê tổng hợp dữ liệu, sự tương quan của dữ liệu và trực quan hóa dữ liệu.

## **Bài 1. Khám phá điểm thi**

### **1.1 Tạo DataFrame từ dữ liệu điểm thi THPT Quốc gia 2020**

**Giải thích:**
Bước đầu tiên là tải dữ liệu từ tệp CSV vào một Spark DataFrame. Chúng ta sẽ khởi tạo một `SparkSession`, sau đó sử dụng `spark.read.csv` để đọc tệp `diemthi2020.csv`. Tùy chọn `header=True` được sử dụng để dòng đầu tiên của tệp được coi là tên cột, và `inferSchema=True` để Spark tự động suy ra kiểu dữ liệu phù hợp cho mỗi cột.

In [ ]:
# Import các thư viện cần thiết
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.functions import col, sum
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("PhanTichDiemThi2020").getOrCreate()

# Đường dẫn đến tệp dữ liệu
file_path = 'phantichdulieu/diemthi2020.csv'

# Đọc dữ liệu từ tệp CSV vào DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Cache DataFrame để tăng tốc độ cho các lần truy cập sau
df.cache()

# Hiển thị 5 dòng đầu tiên và schema của DataFrame
print("5 dòng đầu tiên của dữ liệu:")
df.show(5)
print("Cấu trúc của DataFrame:")
df.printSchema()

**Nhận xét:**
DataFrame đã được tạo thành công. Các cột điểm đã được tự động nhận dạng với kiểu `double`, phù hợp cho việc tính toán. Dữ liệu đã sẵn sàng cho các bước phân tích tiếp theo.

### **1.2 Thống kê sơ bộ về điểm thi các môn**

**Giải thích:**
Chúng ta sẽ sử dụng phương thức `describe()` trên DataFrame để nhận các thông tin thống kê cơ bản như số lượng, trung bình, độ lệch chuẩn, giá trị nhỏ nhất và lớn nhất cho tất cả các cột điểm thi. Điều này giúp có cái nhìn tổng quan ban đầu về dữ liệu.

In [ ]:
diem_thi_cols = [c for c in df.columns if c not in ['so_bao_danh', 'Ma_tinh']]
df.select(diem_thi_cols).describe().show()

**Nhận xét:**
Bảng thống kê cho thấy số lượng thí sinh dự thi các môn là khác nhau. Điểm trung bình và độ lệch chuẩn cũng có sự chênh lệch giữa các môn, phản ánh độ khó và mức độ phân hóa điểm của từng môn thi.

### **1.3 Tính độ lệch, độ nhọn của phân bố điểm thi 3 môn: Toán, Văn, Ngoại ngữ**

**Giải thích:**
Để hiểu sâu hơn về hình dạng phân bố điểm, chúng ta sẽ tính độ lệch (skewness) và độ nhọn (kurtosis). Độ lệch cho biết mức độ đối xứng của phân bố, trong khi độ nhọn cho biết mức độ tập trung của dữ liệu quanh giá trị trung tâm.

In [ ]:
mon_hoc_3 = ['Toan', 'Ngu_van', 'Ngoai_ngu']
exprs = [F.skewness(col).alias(f"{col}_skewness") for col in mon_hoc_3] + \
        [F.kurtosis(col).alias(f"{col}_kurtosis") for col in mon_hoc_3]
df.select(mon_hoc_3).na.drop().agg(*exprs).show()

**Nhận xét:**
Các giá trị skewness âm cho thấy phân bố điểm của cả 3 môn đều lệch trái, tức là có nhiều thí sinh đạt điểm cao. Giá trị kurtosis dương cho thấy phân bố điểm nhọn hơn phân phối chuẩn, tức là điểm thi tập trung nhiều quanh giá trị trung bình.

### **1.4 Vẽ biểu đồ tần suất của từng điểm thi**

**Giải thích:**
Chúng ta sẽ vẽ biểu đồ tần suất (histogram) cho điểm các môn Toán, Văn, Ngoại ngữ theo 3 cách khác nhau để so sánh hiệu năng và kết quả: 
a) Tính toán song song trên Spark, chỉ lấy kết quả tổng hợp về Driver.
b) Kéo toàn bộ dữ liệu về Driver rồi vẽ.
c) Lấy một mẫu 2% dữ liệu rồi vẽ.

In [ ]:
print("--- Cách a) Gộp dữ liệu trên workers rồi thống kê từng nhóm ở Drive ---")
start_time_a = time.time()
for mon in mon_hoc_3:
    binned_df = df.select(mon).na.drop().withColumn('bin', (F.col(mon) * 2).cast('int') / 2)
    counts = binned_df.groupBy('bin').count().orderBy('bin').collect()
    x_axis = [row['bin'] for row in counts]; y_axis = [row['count'] for row in counts]
    plt.figure(figsize=(12, 6)); plt.bar(x_axis, y_axis, width=0.4); plt.title(f'Biểu đồ tần suất điểm môn {mon} (Cách a)'); plt.show()
print(f"Thời gian thực thi (Cách a): {time.time() - start_time_a:.2f} giây\n")

print("--- Cách b) Lấy toàn bộ dữ liệu về Drive để vẽ biểu đồ tần suất ---")
start_time_b = time.time()
for mon in mon_hoc_3:
    scores = df.select(mon).na.drop().rdd.flatMap(lambda x: x).collect()
    plt.figure(figsize=(12, 6)); plt.hist(scores, bins=np.arange(0, 10.5, 0.5), edgecolor='black'); plt.title(f'Biểu đồ tần suất điểm môn {mon} (Cách b)'); plt.show()
print(f"Thời gian thực thi (Cách b): {time.time() - start_time_b:.2f} giây\n")

print("--- Cách c) Lấy mẫu 2% theo điểm thi rồi thống kê và vẽ biểu đồ ---")
start_time_c = time.time()
df_sample = df.sample(False, 0.02, seed=42)
for mon in mon_hoc_3:
    scores_sample = df_sample.select(mon).na.drop().rdd.flatMap(lambda x: x).collect()
    plt.figure(figsize=(12, 6)); plt.hist(scores_sample, bins=np.arange(0, 10.5, 0.5), edgecolor='black'); plt.title(f'Biểu đồ tần suất điểm môn {mon} (Cách c - Mẫu 2%)'); plt.show()
print(f"Thời gian thực thi (Cách c): {time.time() - start_time_c:.2f} giây\n")

**Nhận xét:**
- **Mức độ tương đồng:** Cách (a) và (b) cho kết quả giống hệt. Cách (c) cho biểu đồ có hình dạng tương tự nhưng kém mịn hơn do lấy mẫu.
- **Thời gian xử lý:** Cách (a) là hiệu quả nhất vì tận dụng được xử lý phân tán của Spark. Cách (b) chậm nhất và không phù hợp với dữ liệu lớn. Cách (c) nhanh nhất nhưng chỉ cho kết quả xấp xỉ.

### **1.5 Liệt kê SBD và điểm của những trường hợp outlier cho điểm thi từng môn**

**Giải thích:**
Chúng ta sẽ sử dụng phương pháp IQR (Interquartile Range) để xác định các điểm ngoại lệ (outliers). Một điểm được coi là outlier nếu nó nhỏ hơn Q1 - 1.5*IQR hoặc lớn hơn Q3 + 1.5*IQR. Chúng ta sẽ lặp qua tất cả các môn để tìm.

In [ ]:
for mon in diem_thi_cols:
    quantiles = df.select(mon).na.drop().approxQuantile(mon, [0.25, 0.75], 0.01)
    Q1, Q3 = quantiles[0], quantiles[1]
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR; upper_bound = Q3 + 1.5 * IQR
    outliers_df = df.filter((col(mon) < lower_bound) | (col(mon) > upper_bound)).select('so_bao_danh', mon)
    print(f"\n--- Outliers môn {mon} (Ngoài khoảng [{lower_bound:.2f}, {upper_bound:.2f}]) ---")
    outliers_df.show(5)

**Nhận xét:**
Kết quả cho thấy hầu hết các outliers là các điểm thi rất thấp. Điều này giúp xác định các trường hợp thí sinh có kết quả bất thường so với mặt bằng chung.

### **1.6 Vẽ biểu đồ hộp (Boxplot) cho điểm thi các môn**

**Giải thích:**
Biểu đồ hộp là một cách hiệu quả để trực quan hóa phân bố dữ liệu, bao gồm các giá trị trung vị, tứ phân vị và các outliers. Chúng ta sẽ lấy một mẫu dữ liệu và sử dụng Matplotlib/Seaborn để vẽ.

In [ ]:
diem_thi_pd_df = df.select(diem_thi_cols).sample(False, 0.1, seed=42).toPandas()
plt.figure(figsize=(16, 8))
sns.boxplot(data=diem_thi_pd_df)
plt.title('Biểu đồ hộp phân bố điểm thi các môn')
plt.ylabel('Điểm thi')
plt.xticks(rotation=45)
plt.show()

**Nhận xét:**
Biểu đồ hộp cho thấy rõ sự khác biệt trong phân bố điểm giữa các môn. Ví dụ, môn GDCD có điểm trung vị rất cao và phân bố hẹp, trong khi các môn khác có sự phân tán rộng hơn.

### **1.7 Tính độ tương quan điểm thi từng cặp môn**

**Giải thích:**
Chúng ta sẽ tính ma trận tương quan Pearson để đo lường mối quan hệ tuyến tính giữa các cặp môn thi. Để làm điều này trong Spark, chúng ta cần gộp các cột điểm thành một cột vector duy nhất bằng `VectorAssembler`, sau đó sử dụng `Correlation.corr`.

In [ ]:
df_corr = df.select(diem_thi_cols).na.drop()
assembler = VectorAssembler(inputCols=diem_thi_cols, outputCol="features")
df_vector = assembler.transform(df_corr).select("features")
matrix = Correlation.corr(df_vector, "features").head()
corr_matrix_pd = pd.DataFrame(matrix[0].toArray(), index=diem_thi_cols, columns=diem_thi_cols)

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_pd, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Ma trận tương quan điểm thi các môn')
plt.show()

**Nhận xét:**
Ma trận tương quan cho thấy các môn trong cùng khối khoa học tự nhiên (Toán, Lý, Hóa) và khoa học xã hội (Văn, Sử, Địa) có tương quan dương mạnh với nhau. Tương quan giữa các môn khác khối thì yếu hơn.

### **1.8 Vẽ biểu đồ phân tán thể hiện phân bố điểm tương quan giữa hai môn Toán và Văn**

**Giải thích:**
Để trực quan hóa mối tương quan giữa điểm Toán và Văn, chúng ta sẽ vẽ biểu đồ phân tán (scatter plot). Mỗi điểm trên biểu đồ đại diện cho một thí sinh.

In [ ]:
pd_sample_scatter = df.select('Toan', 'Ngu_van').na.drop().sample(False, 0.02, seed=42).toPandas()
plt.figure(figsize=(10, 8))
plt.scatter(pd_sample_scatter['Toan'], pd_sample_scatter['Ngu_van'], alpha=0.5, s=5)
plt.title('Biểu đồ phân tán điểm Toán và Ngữ văn')
plt.xlabel('Điểm Toán'); plt.ylabel('Điểm Ngữ văn')
plt.grid(True); plt.show()

**Nhận xét:**
Biểu đồ phân tán cho thấy một mối tương quan dương nhẹ giữa điểm Toán và Văn, phù hợp với hệ số tương quan đã tính toán trước đó. Các điểm tập trung đông nhất ở khu vực 6-9 điểm cho cả hai môn.

## **Bài 2. Khám phá tương quan điểm giữa các tỉnh**

### **2.1 Tìm những bài thi môn toán nằm ngoài khoảng dữ liệu hợp lệ theo từng tỉnh**

**Giải thích:**
Chúng ta sẽ áp dụng phương pháp IQR để tìm outliers môn Toán, nhưng lần này sẽ tính toán riêng cho từng tỉnh bằng cách sử dụng Window Functions trong Spark. Điều này giúp xác định thí sinh có điểm bất thường so với mặt bằng chung của tỉnh nhà.

In [ ]:
windowSpec = Window.partitionBy('Ma_tinh')
df_with_bounds = df.withColumn("Q1_Toan", F.percentile_approx("Toan", 0.25).over(windowSpec)) \
                     .withColumn("Q3_Toan", F.percentile_approx("Toan", 0.75).over(windowSpec))
df_with_bounds = df_with_bounds.withColumn("IQR_Toan", col('Q3_Toan') - col('Q1_Toan'))
df_with_bounds = df_with_bounds.withColumn("lower_bound_Toan", col('Q1_Toan') - 1.5 * col('IQR_Toan')) \
                               .withColumn("upper_bound_Toan", col('Q3_Toan') + 1.5 * col('IQR_Toan'))
outliers_by_province_df = df_with_bounds.filter((col('Toan') < col('lower_bound_Toan')) | (col('Toan') > col('upper_bound_Toan')))
print("Các bài thi môn Toán có điểm ngoại lệ theo từng tỉnh:")
outliers_by_province_df.select('Ma_tinh', 'so_bao_danh', 'Toan', 'lower_bound_Toan', 'upper_bound_Toan').show()

**Nhận xét:**
Kết quả cho thấy các thí sinh có điểm Toán là outlier so với tỉnh của họ. Phân tích này mang lại cái nhìn chi tiết và chính xác hơn về các trường hợp bất thường trong bối cảnh địa phương.

### **2.2 Dùng biểu đồ hộp để minh họa phân bố dữ liệu điểm môn toán của 10 tỉnh**

**Giải thích:**
Để so sánh trực quan phân bố điểm Toán giữa các tỉnh, chúng ta sẽ vẽ biểu đồ hộp cho 10 tỉnh có số lượng thí sinh đông nhất.

In [ ]:
top_10_provinces = df.filter(col('Toan').isNotNull()).groupBy('Ma_tinh').count().orderBy('count', ascending=False).limit(10).select('Ma_tinh').rdd.flatMap(lambda x: x).collect()
pd_top10_toan = df.filter(df['Ma_tinh'].isin(top_10_provinces)).select('Ma_tinh', 'Toan').na.drop().sample(False, 0.5, seed=42).toPandas()
plt.figure(figsize=(18, 10)); sns.boxplot(x='Ma_tinh', y='Toan', data=pd_top10_toan, order=top_10_provinces); plt.title('Phân bố điểm môn Toán của 10 tỉnh đông thí sinh nhất'); plt.show()

**Nhận xét:**
Biểu đồ hộp cho thấy sự khác biệt rõ rệt về điểm trung vị và độ phân tán môn Toán giữa các tỉnh, nhấn mạnh tầm quan trọng của việc phân tích dữ liệu trong bối cảnh địa phương.

### **2.3. Tổng quát, tìm những điểm ngoại lệ của thí sinh từng tỉnh ở tất cả các môn**

**Giải thích:**
Chúng ta sẽ mở rộng phương pháp ở mục 2.1 để tự động tìm các điểm ngoại lệ cho tất cả các môn thi, dựa trên phân bố điểm của từng tỉnh.

In [ ]:
for mon in diem_thi_cols:
    windowSpec = Window.partitionBy('Ma_tinh')
    df_bounds_mon = df.withColumn(f"Q1_{mon}", F.percentile_approx(mon, 0.25).over(windowSpec)).withColumn(f"Q3_{mon}", F.percentile_approx(mon, 0.75).over(windowSpec))
    df_bounds_mon = df_bounds_mon.withColumn(f"IQR_{mon}", col(f'Q3_{mon}') - col(f'Q1_{mon}'))
    df_bounds_mon = df_bounds_mon.withColumn(f"lower_bound_{mon}", col(f'Q1_{mon}') - 1.5 * col(f'IQR_{mon}')).withColumn(f"upper_bound_{mon}", col(f'Q3_{mon}') + 1.5 * col(f'IQR_{mon}'))
    outliers_mon_df = df_bounds_mon.filter((col(mon) < col(f'lower_bound_{mon}')) | (col(mon) > col(f'upper_bound_{mon}')))
    print(f"--- Các điểm ngoại lệ của môn {mon} theo tỉnh ---")
    outliers_mon_df.select('Ma_tinh', 'so_bao_danh', mon).show(5)

**Nhận xét:**
Quy trình này đã tự động hóa việc phát hiện các trường hợp thi cử có điểm bất thường ở cấp độ địa phương trên toàn bộ các môn học, rất hữu ích cho việc giám sát chất lượng.

## **Bài 3. Thống kê dữ liệu theo khối**

### **3.1 Vẽ biểu đồ tần suất phân bố điểm từng khối A, B, C, D**

**Giải thích:**
Chúng ta sẽ tính tổng điểm cho các khối A, B, C, D và vẽ biểu đồ tần suất để so sánh phổ điểm giữa các khối.

In [ ]:
khoi_a_cols = ['Toan', 'Vat_li', 'Hoa_hoc']; khoi_b_cols = ['Toan', 'Hoa_hoc', 'Sinh_hoc']
khoi_c_cols = ['Ngu_van', 'Lich_su', 'Dia_li']; khoi_d_cols = ['Toan', 'Ngu_van', 'Ngoai_ngu']
df_khoi = df.withColumn('Khoi_A', sum(col(c) for c in khoi_a_cols)).withColumn('Khoi_B', sum(col(c) for c in khoi_b_cols)) \
            .withColumn('Khoi_C', sum(col(c) for c in khoi_c_cols)).withColumn('Khoi_D', sum(col(c) for c in khoi_d_cols))
khoi_list = ['Khoi_A', 'Khoi_B', 'Khoi_C', 'Khoi_D']
for khoi in khoi_list:
    scores = df_khoi.select(khoi).na.drop().rdd.flatMap(lambda x: x).collect()
    plt.figure(figsize=(12, 6)); plt.hist(scores, bins=30, range=(0, 30), edgecolor='black'); plt.title(f'Biểu đồ tần suất phân bố điểm {khoi}'); plt.show()

**Nhận xét:**
Các biểu đồ cho thấy khối A và B có điểm tập trung ở mức cao, thể hiện tính cạnh tranh cao. Khối C và D có phổ điểm rộng hơn và đỉnh phân bố thấp hơn.

### **3.2 Tính độ lệch, độ nhọn của phân bố điểm thi các khối thi**

**Giải thích:**
Chúng ta sẽ tính skewness và kurtosis cho tổng điểm các khối để có thông tin định lượng về hình dạng phân bố.

In [ ]:
khoi_exprs = [F.skewness(k).alias(f"{k}_skewness") for k in khoi_list] + [F.kurtosis(k).alias(f"{k}_kurtosis") for k in khoi_list]
df_khoi.select(khoi_list).na.drop().agg(*khoi_exprs).show()

**Nhận xét:**
Tất cả các khối đều có skewness âm (lệch trái, nhiều điểm cao) và kurtosis dương (phân bố nhọn), xác nhận lại quan sát từ biểu đồ tần suất.

### **3.3 Vẽ biểu đồ tròn biểu diễn tỷ lệ thí sinh khối A, B, C, D từ 15 điểm trở lên**

**Giải thích:**
Chúng ta sẽ lọc những thí sinh có tổng điểm từ 15 trở lên cho mỗi khối, sau đó đếm số lượng và vẽ biểu đồ tròn để xem tỷ lệ giữa các khối.

In [ ]:
counts = [df_khoi.filter(col(k) >= 15).count() for k in khoi_list]
labels = ['Khối A', 'Khối B', 'Khối C', 'Khối D']
plt.figure(figsize=(10, 8)); plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90, shadow=True); plt.title('Tỷ lệ thí sinh các khối đạt từ 15 điểm trở lên'); plt.axis('equal'); plt.show()

**Nhận xét:**
Biểu đồ cho thấy khối A chiếm tỷ lệ lớn nhất trong số các thí sinh đạt từ 15 điểm trở lên, phản ánh đây là khối thi có số lượng thí sinh chất lượng cao đông đảo nhất.